In [1]:
import sys
sys.path.append('/home/wangs/rs/lib')
import ff
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from multiprocessing import Pool
from tqdm import tqdm
import statsmodels.api as sm
from datetime import datetime
import mplfinance as mpf
from functools import partial
import matplotlib as mpl

In [2]:
from gplearn.functions import _protected_sqrt, make_function
from gplearn.genetic import SymbolicRegressor, SymbolicTransformer
from gplearn.genetic import SymbolicClassifier
from gplearn import fitness
from gplearn.fitness import make_fitness

初始化种群：将因子的计算公式表示成树形的结构，预先设置函数集和指标集，进行随机组合生成一系列因子表达式，作为第一代种群集合

In [4]:
# 截面分位数
def _rank(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        return np.nan_to_num((data_df.rank()/data_df.count()).values.reshape(-1))
    except:
        return np.zeros(len(data))

# 两组数据对应位置上的较大值
def _max(data1,data2):  
    try:
        return np.maximum(data1,data2)
    except:
        return np.zeros(len(data1))

# 两组数据对应位置上的较小值
def _min(data1,data2):  
    try:
        return np.minimum(data1,data2)
    except:
        return np.zeros(len(data1))

# 两组数据截面分位数之和
def _rank_add(data1,data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        rank1 = data1_df.rank()/data1_df.count()
        rank2 = data2_df.rank()/data2_df.count()
        value = rank1+rank2
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# 两组数据截面分位数之差
def _rank_sub(data1,data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        rank1 = data1_df.rank()/data1_df.count()
        rank2 = data2_df.rank()/data2_df.count()
        value = rank1-rank2
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# 两组数据截面分位数乘积
def _rank_mul(data1,data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        rank1 = data1_df.rank()/data1_df.count()
        rank2 = data2_df.rank()/data2_df.count()
        value = rank1*rank2
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# 两组数据截面分位数比值
def _rank_div(data1,data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        rank1 = data1_df.rank()/data1_df.count()
        rank2 = data2_df.rank()/data2_df.count()
        value = rank1/rank2
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# shift 1
def _delay1(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        return np.nan_to_num(data_df.shift(1,axis=1).values.reshape(-1))
    except:
        return np.zeros(len(data))

# shift 5
def _delay5(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        return np.nan_to_num(data_df.shift(5,axis=1).values.reshape(-1))
    except:
        return np.zeros(len(data))

# diff 1
def _delta1(data):  
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        return np.nan_to_num(data_df.diff(1,axis=1).values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 5天取std
def _ts_std5(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value=data_df.rolling(5,min_periods=2,axis=1).std()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 10天取std
def _ts_std10(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value=data_df.rolling(10,min_periods=5,axis=1).std()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 5天取最小值
def _ts_min5(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value=data_df.rolling(5,min_periods=2,axis=1).min()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 10天取最小值
def _ts_min10(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value=data_df.rolling(10,min_periods=5,axis=1).min()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 5天取最大值
def _ts_max5(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value=data_df.rolling(5,min_periods=2,axis=1).max()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 10天取最大值
def _ts_max10(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value=data_df.rolling(10,min_periods=5,axis=1).max()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 10天取最小值
def _ts_cov10(data1, data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        value=data1_df.T.rolling(window=10, min_periods=5).cov(data2_df.T).T
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# 时序协方差
def _ts_cov5(data1, data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        value=data1_df.T.rolling(window=5, min_periods=2).cov(data2_df.T).T
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

def _ts_corr10(data1, data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        value=data1_df.T.rolling(window=10, min_periods=5).corr(data2_df.T).T
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# 时序相关系数
def _ts_corr5(data1, data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        value=data1_df.T.rolling(window=5, min_periods=2).corr(data2_df.T).T
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# 时序 mean/std
def _ts_ms10(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        roll = data_df.rolling(window=10, min_periods=5,axis=1)
        value = roll.mean() / roll.std()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))
    
def _ts_ms5(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        roll = data_df.rolling(window=5, min_periods=2,aixs=1)
        value = roll.mean() / roll.std()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# 时序z分数
def _ts_zscore10(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        roll = data_df.rolling(window=10, min_periods=5,axis=1)
        value = (data_df - roll.mean())/roll.std()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

def _ts_zscore5(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        roll = data_df.rolling(window=5, min_periods=2,axis=1)
        value = (data_df - roll.mean())/roll.std()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# 时序pct_change
def _ts_chg10(data):
    try:
        data_df = pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value = data_df.pct_change(10, axis=1)
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))
    
def _ts_chg5(data):
    try:
        data_df = pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value = data_df.pct_change(5, axis=1)
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))
    
def _ts_chg1(data):
    try:
        data_df = pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value = data_df.pct_change(1, axis=1)
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# 时序偏度
def _ts_skew5(data):
    try:
        data_df = pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value = data_df.rolling(5,min_periods=2,axis=1).skew()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))
    
def _ts_skew10(data):
    try:
        data_df = pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value = data_df.rolling(10,min_periods=5,axis=1).skew()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

rank=make_function(function=_rank,name='rank',arity=1)
max_=make_function(function=_max,name='max_',arity=2)
min_=make_function(function=_min,name='min_',arity=2)
rank_add=make_function(function=_rank_add,name='rank_add',arity=2)
rank_sub=make_function(function=_rank_sub,name='rank_sub',arity=2)
rank_mul=make_function(function=_rank_mul,name='rank_mul',arity=2)
rank_div=make_function(function=_rank_div,name='rank_div',arity=2)
delay1=make_function(function=_delay1,name='delay1',arity=1)
delay5=make_function(function=_delay5,name='delay5',arity=1)
delta1=make_function(function=_delta1,name='delta1',arity=1)
ts_std5=make_function(function=_ts_std5,name='ts_std5',arity=1)
ts_std10=make_function(function=_ts_std10,name='ts_std10',arity=1)
ts_min5=make_function(function=_ts_min5,name='ts_min5',arity=1)
ts_min10=make_function(function=_ts_min10,name='ts_min10',arity=1)
ts_max5=make_function(function=_ts_max5,name='ts_max5',arity=1)
ts_max10=make_function(function=_ts_max10,name='ts_max10',arity=1)
ts_cov10=make_function(function=_ts_cov10,name='ts_cov10',arity=2)
ts_cov5=make_function(function=_ts_cov5,name='ts_cov5',arity=2)
ts_corr10=make_function(function=_ts_corr10,name='ts_corr10',arity=2)
ts_corr5=make_function(function=_ts_corr5,name='ts_corr5',arity=2)
ts_ms10=make_function(function=_ts_ms10,name='ts_ms10',arity=1)
ts_ms5=make_function(function=_ts_ms5,name='ts_ms5',arity=1)
ts_zscore10=make_function(function=_ts_zscore10,name='ts_zscore10',arity=1)
ts_zscore5=make_function(function=_ts_zscore5,name='ts_zscore5',arity=1)
ts_chg10=make_function(function=_ts_chg10,name='ts_chg10',arity=1)
ts_chg5=make_function(function=_ts_chg5,name='ts_chg5',arity=1)
ts_chg1=make_function(function=_ts_chg1,name='ts_chg1',arity=1)
ts_skew10=make_function(function=_ts_skew10,name='ts_skew10',arity=1)
ts_skew5=make_function(function=_ts_skew5,name='ts_skew5',arity=1)

In [25]:
import statsmodels.api as sm

def rankIC_metric(y,y_pred,w=None):
    
    def cal_rankIC(df):
        df.index=df.code
        # 去极值
        M=df.y_pred.mean()    #df.y_pred.median()
        M1=df.y_pred.std()    #(df.y_pred-M).abs().median()
        df.y_pred.loc[df.y_pred>M+3*M1]=M+3*M1    #5
        df.y_pred.loc[df.y_pred<M-3*M1]=M-3*M1
        # 行业市值中性化
        day=df.date.iloc[0]
        ind=inds[day].copy()
        ind['mkv']=df.loc[ind.index,'mkv']
        input_x=ind.dropna().astype(float)
        input_y=df.loc[input_x.index,'y_pred']
        y_true=df.loc[input_x.index,'y'].values
        if len(input_x)==0:
            return 0
        else:
            mod=sm.OLS(input_y,input_x)
            res=mod.fit()
            y_adj=res.resid
            #标准化
            y_adj=(y_adj-y_adj.mean())/y_adj.std()
        
            if len(np.unique(y_adj.values)) <=2:
                return 0
            else:
                #return y_adj.corr(input_xy.mkt_delay,method='spearman')
                return spearmanr(np.nan_to_num(y_adj.values),np.nan_to_num(y_true))[0]
            
    if len(np.unique(y_pred))<=2:
        return 0
    else:
        try:
            data=pd.DataFrame({'y_pred':y_pred,'y':y,'mkv':np.log(mkv_s),'code':test_code,'date':test_date})
            res=data.groupby('date').apply(cal_rankIC)
            return np.abs(np.nanmean(res))
        except:
            return 0

In [26]:
function_set = ['add','sub','mul','div','sqrt','log','inv','abs','neg',rank,max_,min_,rank_add,rank_sub,rank_mul,rank_div,delay1,delay5,delta1,ts_std5,ts_std10,ts_min5,ts_min10,ts_max5,ts_max10,ts_cov10,ts_cov5,ts_corr10,ts_corr5,ts_ms10,ts_ms5,ts_zscore10,ts_zscore5,ts_chg10,ts_chg5,ts_chg1,ts_skew10,ts_skew5]

In [27]:
stock_feature = ['accelerated_turnover_rank_RC','CSK_XYY_UP_DOWN_120D_RC','high_fre_vol_RC','high_fre_diff_vol_RC','high_fre_absdiff_vol_RC','peak_count_vol_RC','overnightsmart20_RC','CTR_RC','jumpCTR_RC','turnover_rate_proportion_l','synergy']  #特征集合'extreme_nervous_RC',

In [28]:
fields = stock_feature + ['1','5','20','40','60']

In [40]:
stock_test=pd.DataFrame({'accelerated_turnover_rank_RC':ff.read('accelerated_turnover_rank_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         'CSK_XYY_UP_DOWN_120D_RC':ff.read('CSK_XYY_UP_DOWN_120D_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         'high_fre_vol_RC':ff.read('high_fre_vol_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         'high_fre_diff_vol_RC':ff.read('high_fre_diff_vol_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         'high_fre_absdiff_vol_RC':ff.read('high_fre_absdiff_vol_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         'peak_count_vol_RC':ff.read('peak_count_vol_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         'overnightsmart20_RC':ff.read('overnightsmart20_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         'CTR_RC': ff.read('CTR_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         'jumpCTR_RC': ff.read('jumpCTR_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         'turnover_rate_proportion_l': ff.read('turnover_rate_proportion_l').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         'synergy':ff.read('synergy').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1)
                        })
#'extreme_nervous_RC':ff.read('extreme_nervous_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),

In [41]:
stock_code = list(np.repeat(list(ff.read('synergy').index),len(ff.read('synergy').loc[:,'20200207':'20210101'].columns)))
date_code = list(np.tile(list(ff.read('synergy').loc[:,'20200207':'20210101'].columns),len(list(ff.read('synergy').index))))

In [42]:
stock_test['code'] = stock_code
stock_test['date'] = date_code
stock_test['1'],stock_test['5'],stock_test['10'],stock_test['20'],stock_test['40'],stock_test['60']=1,5,10,20,40,60

In [56]:
close = ff.read('close').copy().loc[ff.read('synergy').index,:]
mkt10=close.shift(-10,axis=1)/close - 1

In [57]:
test_mkt=mkt10.loc[:,'20200207':'20210101'].values.reshape(-1)

In [53]:
rankIC=make_fitness(function=rankIC_metric,greater_is_better=True)

In [54]:
test_gp=SymbolicTransformer(feature_names = fields,
                            function_set = function_set, #所有算子
                            generations = 3, #进化代数
                            population_size = 100, #种群规模
                            tournament_size = 10, #竞标赛规模
                            p_crossover=0.4,
                            p_subtree_mutation=0.05,
                            p_hoist_mutation=0.01,
                            p_point_mutation=0.03,
                            p_point_replace=0.35,
                            init_depth=(1,4),
                            const_range = None,
                            metric=rankIC,
                            parsimony_coefficient = 'auto',
                            low_memory=True,
                            verbose=2,
                            n_jobs = 6)

In [55]:
test_gp.fit(np.nan_to_num(stock_test.loc[:,fields].values),np.nan_to_num(test_mkt))

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    1.7s remaining:    3.5s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    1.9s finished


   0     5.31                0        6                0              N/A      3.95s


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    1.4s remaining:    2.9s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    2.6s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


   1     5.82                0        9                0              N/A      2.67s


[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    1.8s remaining:    3.6s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    3.6s finished


   2     6.17                0        9                0              N/A      0.00s


SymbolicTransformer(const_range=None,
                    feature_names=['accelerated_turnover_rank_RC',
                                   'CSK_XYY_UP_DOWN_120D_RC', 'high_fre_vol_RC',
                                   'high_fre_diff_vol_RC',
                                   'high_fre_absdiff_vol_RC',
                                   'peak_count_vol_RC', 'overnightsmart20_RC',
                                   'CTR_RC', 'jumpCTR_RC',
                                   'turnover_rate_proportion_l', 'synergy', '1',
                                   '5', '20', '40', '60'],
                    function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'l...
                                  <gplearn.functions._Function object at 0x7faa05e04e20>,
                                  <gplearn.functions._Function object at 0x7faa05eb6ca0>, ...],
                    generations=3, init_depth=(1, 4), low_memory=True,
                    metric=<gplearn.fitness._Fitness object at 0x7fa9313eacd0>,
                    n_jobs=6, p_crossover=0.4, p_point_mutation=0.03,
                    p_point_replace=0.35, p_subtree_mutation=0.05,
                    parsimony_coefficient='auto', population_size=100,
                    tournament_size=10, verbose=2)

In [38]:
fields

['accelerated_turnover_rank_RC',
 'CSK_XYY_UP_DOWN_120D_RC',
 'high_fre_vol_RC',
 'high_fre_diff_vol_RC',
 'high_fre_absdiff_vol_RC',
 'peak_count_vol_RC',
 'overnightsmart20_RC',
 'CTR_RC',
 'jumpCTR_RC',
 'turnover_rate_proportion_l',
 'synergy',
 '1',
 '5',
 '20',
 '40',
 '60']

In [18]:
function_set

['add',
 'sub',
 'mul',
 'div',
 'sqrt',
 'log',
 'inv',
 'abs',
 'neg',

In [61]:
stock_test.loc[:,fields]

,accelerated_turnover_rank_RC,CSK_XYY_UP_DOWN_120D_RC,high_fre_vol_RC,high_fre_diff_vol_RC,high_fre_absdiff_vol_RC,peak_count_vol_RC,overnightsmart20_RC,CTR_RC,jumpCTR_RC,turnover_rate_proportion_l,synergy,1,5,20,40,60
0,-1594.0,0.124985,0.608404,0.867604,0.510132,15.0,-0.789820,-6.434608,-6.908734,-0.000000,0.331655,1,5,20,40,60
1,-2554.0,0.122099,0.579994,0.877081,0.514625,13.0,-0.796881,-6.472201,-7.117582,-0.023756,0.338297,1,5,20,40,60
2,-3021.0,0.183596,0.586373,0.851523,0.507701,17.0,-0.813100,-6.959742,-6.539672,-0.017686,0.341056,1,5,20,40,60
3,-2664.0,0.159378,0.575233,0.845270,0.504572,5.0,-0.807194,-6.797148,-6.460341,-0.009009,0.337031,1,5,20,40,60
4,-2638.0,0.227376,0.558234,0.851457,0.508584,16.0,-0.774690,-6.298431,-6.278101,-0.002533,0.338436,1,5,20,40,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.173521,1,5,20,40,60
1169854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.175424,1,5,20,40,60
1169855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.174848,1,5,20,40,60
1169856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.174848,1,5,20,40,60


In [60]:
test_mkt

array([ 0.06566347,  0.05034483,  0.01690331, ..., -0.13342898,
       -0.09638205, -0.09719888])

In [62]:
ff.read('synergy')

,20200102,20200103,20200106,20200107,20200108,20200109,20200110,20200113,20200114,20200115,...,20240408,20240409,20240410,20240411,20240412,20240415,20240416,20240417,20240418,20240419
000001.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.284755,0.281048,0.266407,0.279347,0.272885,0.267183,0.264040,0.260250,0.258510,0.279172
000002.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.341264,0.341944,0.332415,0.329310,0.320903,0.322821,0.322309,0.322644,0.329383,0.338925
000004.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.295437,0.297834,0.295403,0.301065,0.303591,0.315650,0.326367,0.338461,0.340177,0.341154
000005.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000006.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.265308,0.270457,0.264737,0.280409,0.277310,0.290069,0.289962,0.294087,0.301289,0.300946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603157.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603555.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603996.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688086.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
